In [ ]:
import os
from pathlib import Path
from os import listdir
from os.path import isfile, join

from PIL import Image
import matplotlib.pyplot as plt

import math
import numpy as np

import tensorflow as tf
import torch
import lpips

from utils.srgan_tools import resolve_single
from utils.srgan import generator
from utils.niqe import niqe
from utils.tools import load_image, plot_sample
from skimage.metrics import structural_similarity

In [ ]:
pre = generator(num_res_blocks=8, batch_size=16, return_features=True)
pre.load_weights('weights/srgan/pre_generator_small.h5')

In [ ]:
pre

In [ ]:
gan = generator(num_res_blocks=8, batch_size=16, return_features=True)
gan.load_weights('weights/srgan/gan_generator_gridsearch_2022-07-26_16_43_06.2346946.h5')

In [ ]:
new = generator(num_res_blocks=8, batch_size=16, return_features=True)

In [ ]:
def mix_model_weights(mp, mg, p=0.5):
    
    mix_w = []
    for i in range(len(mp.layers)):
        pre_w = mp.layers[i].get_weights()
        gan_w = mg.layers[i].get_weights()
        #print(mp.layers[i].name, len(pre_w))
        
        mix_w.append(mix_layer_weights(pre_w, gan_w, p))

        
    return mix_w

In [ ]:
def mix_layer_weights(wp, wg, p=0.5):
    weights = []
    for i in range(len(wp)):
        w = p * wp[i] + (1-p) * wg[i]
        weights.append(w)
    return weights

In [ ]:
for p in range(0,101,10):
    mix_w = mix_model_weights(pre, gan, p/100.0)

    for i in range(len(new.layers)):
        new.layers[i].set_weights(mix_w[i])
    
    new.save_weights(f'weights/srgan/small_interpolation_{p}.h5')

In [ ]:
head = generator(scale=4,
                 num_filters=64, 
                 num_res_blocks=8,
                 batch_size=16,
                 batch_norm=False,
                 activation='relu',
                 upsampling='TransposeConv',
                 head_only=True)
head.set_weights(pre.get_weights()[-4:]) 

In [ ]:
for i in range(len(head.get_weights())):
    print((head.get_weights()[i] == pre.get_weights()[-4:][i]).all())

In [ ]:
pre.get_weights()[-4:]